In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

: 

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'projects/'

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate